# 从 ConversationBufferWindowMemory 或 ConversationTokenBufferMemory 迁移

如果您正尝试从以下旧内存类迁移，请遵循本指南：

| 内存类型                      | 描述                                                                                                                                                       |
|----------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `ConversationBufferWindowMemory` | 保留对话的最后 `n` 条消息。当消息数超过 `n` 条时，将丢弃最旧的消息。                                                                                                               |
| `ConversationTokenBufferMemory`  | 仅保留对话中最近的消息，但前提是对话中的总令牌数不超过某个限制。 |

`ConversationBufferWindowMemory` 和 `ConversationTokenBufferMemory` 对原始对话历史进行额外的处理，以将对话历史修剪到适合聊天模型上下文窗口的大小。

此处理功能可以使用 LangChain 内置的 [trim_messages](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html) 函数来完成。

:::important

我们将首先探讨一种将处理逻辑应用于整个对话历史的简单方法。

虽然这种方法易于实现，但有一个缺点：随着对话的进行，延迟也会随之增加，因为在每个回合，逻辑都会重新应用于对话中的所有先前交流。

更高级的策略侧重于增量更新对话历史以避免重复处理。

例如，langgraph [关于对话摘要的操作指南](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/) 演示了如何在丢弃旧消息的同时维护对话的运行摘要，从而确保在后续回合中不会重复处理它们。
:::

## 设置

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-openai langchain

In [1]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

## LLMChain / Conversation Chain 的旧用法

<details open>

In [3]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [
        SystemMessage(content="You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

# highlight-start
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
# highlight-end

legacy_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
    # highlight-next-line
    memory=memory,
)

legacy_result = legacy_chain.invoke({"text": "my name is bob"})
print(legacy_result)

legacy_result = legacy_chain.invoke({"text": "what was my name"})
print(legacy_result)

{'text': 'Nice to meet you, Bob! How can I assist you today?', 'chat_history': []}
{'text': 'Your name is Bob. How can I assist you further, Bob?', 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})]}


</details>

## 重写 ConversationBufferWindowMemory 逻辑

我们首先创建适当的逻辑来处理对话历史，然后展示如何将其集成到应用程序中。之后，您可以根据特定需求将此基本设置替换为更高级的逻辑。

我们将使用 `trim_messages` 来实现仅保留对话中最后 `n` 条消息的逻辑。当消息数量超过 `n` 时，它将丢弃最旧的消息。

此外，如果存在系统消息，我们也会保留它——当存在时，它将是包含聊天模型指令的对话中的第一条消息。

In [4]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_openai import ChatOpenAI

messages = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("why is 42 always the answer?"),
    AIMessage(
        "Because it’s the only number that’s constantly right, even when it doesn’t add up!"
    ),
    HumanMessage("What did the cow say?"),
]

In [5]:
from langchain_core.messages import trim_messages

selected_messages = trim_messages(
    messages,
    token_counter=len,  # <-- len will simply count the number of messages rather than tokens
    max_tokens=5,  # <-- allow up to 5 messages.
    strategy="last",
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # start_on="human" makes sure we produce a valid chat history
    start_on="human",
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    allow_partial=False,
)

for msg in selected_messages:
    msg.pretty_print()

================================ System Message ================================

you're a good assistant, you always respond with a joke.
================================== Ai Message ==================================

Hmmm let me think.

Why, he's probably chasing after the last cup of coffee in the office!
================================ Human Message =================================

why is 42 always the answer?
================================== Ai Message ==================================

Because it’s the only number that’s constantly right, even when it doesn’t add up!
================================ Human Message =================================

What did the cow say?


## 重写 `ConversationTokenBufferMemory` 逻辑

在这里，我们将使用 `trim_messages` 来在对话的总 token 数不超过特定限制的约束下，保留系统消息和对话中最新的消息。

In [6]:
from langchain_core.messages import trim_messages

selected_messages = trim_messages(
    messages,
    # Please see API reference for trim_messages for other ways to specify a token counter.
    token_counter=ChatOpenAI(model="gpt-4o"),
    max_tokens=80,  # <-- token limit
    # The start_on is specified
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # start_on="human" makes sure we produce a valid chat history
    start_on="human",
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    strategy="last",
)

for msg in selected_messages:
    msg.pretty_print()

================================ System Message ================================

you're a good assistant, you always respond with a joke.
================================ Human Message =================================

why is 42 always the answer?
================================== Ai Message ==================================

Because it’s the only number that’s constantly right, even when it doesn’t add up!
================================ Human Message =================================

What did the cow say?


## 现代用法，结合 LangGraph

下面的示例展示了如何使用 LangGraph 添加简单的对话预处理逻辑。

:::note

如果你想避免每次都对整个对话历史进行计算，可以遵循[关于对话摘要的指南](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)，该指南演示了
如何丢弃较早的消息，确保它们在后续回合中不会被重复处理。

:::

<details open>

In [7]:
import uuid

from IPython.display import Image, display
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define a chat model
model = ChatOpenAI()


# Define the function that calls the model
def call_model(state: MessagesState):
    # highlight-start
    selected_messages = trim_messages(
        state["messages"],
        token_counter=len,  # <-- len will simply count the number of messages rather than tokens
        max_tokens=5,  # <-- allow up to 5 messages.
        strategy="last",
        # Most chat models expect that chat history starts with either:
        # (1) a HumanMessage or
        # (2) a SystemMessage followed by a HumanMessage
        # start_on="human" makes sure we produce a valid chat history
        start_on="human",
        # Usually, we want to keep the SystemMessage
        # if it's present in the original history.
        # The SystemMessage has special instructions for the model.
        include_system=True,
        allow_partial=False,
    )

    # highlight-end
    response = model.invoke(selected_messages)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


# Adding memory is straight forward in langgraph!
# highlight-next-line
memory = MemorySaver()

app = workflow.compile(
    # highlight-next-line
    checkpointer=memory
)


# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
# highlight-next-line
config = {"configurable": {"thread_id": thread_id}}

input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
config = {"configurable": {"thread_id": thread_id}}
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! How can I assist you today?
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

Your name is Bob. How can I help you, Bob?


</details>

## 使用预构建的 langgraph agent

此示例展示了如何将 Agent Executor 与使用 [create_tool_calling_agent](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html) 函数构建的预构建 agent 结合使用。

如果您使用的是 [旧版 LangChain 预构建 agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)，您应该能够用新的 [langgraph 预构建 agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) 替换该代码。后者利用了 chat model 原生的工具调用功能，并且开箱即用的效果可能更好。

<details open>

In [8]:
import uuid

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


memory = MemorySaver()
model = ChatOpenAI()


# highlight-start
def prompt(state) -> list[BaseMessage]:
    """Given the agent state, return a list of messages for the chat model."""
    # We're using the message processor defined above.
    return trim_messages(
        state["messages"],
        token_counter=len,  # <-- len will simply count the number of messages rather than tokens
        max_tokens=5,  # <-- allow up to 5 messages.
        strategy="last",
        # Most chat models expect that chat history starts with either:
        # (1) a HumanMessage or
        # (2) a SystemMessage followed by a HumanMessage
        # start_on="human" makes sure we produce a valid chat history
        start_on="human",
        # Usually, we want to keep the SystemMessage
        # if it's present in the original history.
        # The SystemMessage has special instructions for the model.
        include_system=True,
        allow_partial=False,
    )


# highlight-end

app = create_react_agent(
    model,
    tools=[get_user_age],
    checkpointer=memory,
    # highlight-next-line
    prompt=prompt,
)

# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_jsMvoIFv970DhqqLCJDzPKsp)
 Call ID: call_jsMvoIFv970DhqqLCJDzPKsp
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob.


</details>

## LCEL：添加预处理步骤

添加复杂对话管理的最简单方法是在聊天模型前面引入一个预处理步骤，并将完整的对话历史传递给预处理步骤。

这种方法在概念上很简单，并且在许多情况下都适用；例如，如果你使用的是 [RunnableWithMessageHistory](/docs/how_to/message_history/) 而不是包装聊天模型，则用预处理器包装聊天模型。

这种方法明显的缺点是，由于两个原因，随着对话历史的增长，延迟开始增加：

1. 随着对话的变长，可能需要从你用于存储对话历史的任何存储中获取更多数据（如果不是存储在内存中）。
2. 预处理逻辑最终会执行大量冗余计算，重复对话先前步骤中的计算。

:::caution

如果你想使用聊天模型的工具调用功能，请记住在向其添加历史预处理步骤之前，先将工具绑定到模型！

:::

<details open>

In [9]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI()


@tool
def what_did_the_cow_say() -> str:
    """Check to see what the cow said."""
    return "foo"


# highlight-start
message_processor = trim_messages(  # Returns a Runnable if no messages are provided
    token_counter=len,  # <-- len will simply count the number of messages rather than tokens
    max_tokens=5,  # <-- allow up to 5 messages.
    strategy="last",
    # The start_on is specified
    # to make sure we do not generate a sequence where
    # a ToolMessage that contains the result of a tool invocation
    # appears before the AIMessage that requested a tool invocation
    # as this will cause some chat models to raise an error.
    start_on=("human", "ai"),
    include_system=True,  # <-- Keep the system message
    allow_partial=False,
)
# highlight-end

# Note that we bind tools to the model first!
model_with_tools = model.bind_tools([what_did_the_cow_say])

# highlight-next-line
model_with_preprocessor = message_processor | model_with_tools

full_history = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("why is 42 always the answer?"),
    AIMessage(
        "Because it’s the only number that’s constantly right, even when it doesn’t add up!"
    ),
    HumanMessage("What did the cow say?"),
]


# We pass it explicity to the model_with_preprocesor for illustrative purposes.
# If you're using `RunnableWithMessageHistory` the history will be automatically
# read from the source the you configure.
model_with_preprocessor.invoke(full_history).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  what_did_the_cow_say (call_urHTB5CShhcKz37QiVzNBlIS)
 Call ID: call_urHTB5CShhcKz37QiVzNBlIS
  Args:


</details>

如果你需要实现更高效的逻辑，并且目前想使用 `RunnableWithMessageHistory`，那么实现的方式是继承 [BaseChatMessageHistory](https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html) 并为 `add_messages` 定义适当的逻辑（该逻辑不只是简单地追加历史记录，而是重写它）。

除非你有充分的理由来实现此解决方案，否则你应该使用 LangGraph。

## 后续步骤

探索 LangGraph 中的持久化：

* [LangGraph 快速入门教程](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
* [如何为图添加持久化（“内存”）](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
* [如何管理对话历史](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
* [如何为对话历史添加摘要](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)

使用简单的 LCEL 添加持久化（更复杂的用例建议使用 LangGraph）：

* [如何添加消息历史](/docs/how_to/message_history/)

处理消息历史：

* [如何修剪消息](/docs/how_to/trim_messages)
* [如何过滤消息](/docs/how_to/filter_messages/)
* [如何合并消息运行](/docs/how_to/merge_message_runs/)